Gurobi中经常需要对带有不同下标的数据进行组合。Gurobi自带的数据结构能够大大地提高效率:

## Multidict

比如每个学生都有语文、数学、英语成绩。如何来存储这个成绩呢？

In [1]:
import gurobipy as grb

In [2]:
student, chinese, math, english = grb.multidict({
    'student1':[1, 2, 3],
    'student2':[2, 3, 4],
    'student3':[3, 4, 5],
    'student4':[4, 5, 6]
})

In [3]:
print(student) # 打印出字典的键。
print(chinese) # 语文成绩的字典。
print(math) # 数学成绩的字典。

['student1', 'student2', 'student3', 'student4']
{'student1': 1, 'student2': 2, 'student3': 3, 'student4': 4}
{'student1': 2, 'student2': 3, 'student3': 4, 'student4': 5}


## TupleList

tuplelist元祖列表，是tuple和list的组合。也就是list元素的tuple类型。

tuplelist在内部存储上和普通的list是一样的，只是Gurobi在继承list类的基础上添加了select方法。因此，可以把tuplelist看作是list对象，可以使用迭代、添加或删除元素等方法。

In [4]:
t1 = grb.tuplelist([(1, 2), (1, 3), (2, 3), (2, 5)])
print(t1.select(1, '*'))
print(t1.select('*', 3))

<gurobi.tuplelist (2 tuples, 2 values each):
 ( 1 , 2 )
 ( 1 , 3 )
>
<gurobi.tuplelist (2 tuples, 2 values each):
 ( 1 , 3 )
 ( 2 , 3 )
>


In [5]:
t1.append((1, 5))
print(t1.select(1, '*'))

<gurobi.tuplelist (3 tuples, 2 values each):
 ( 1 , 2 )
 ( 1 , 3 )
 ( 1 , 5 )
>


In [6]:
# 查看某个元素是否在被选择的元祖中。
if((1,2) in t1.select(1, '*')):
    print(True)
else:
    print(False)

True


## Tupledict

tupledict是dict的子类。能够帮助我们高效地操作gurobi中的变量子集。

比如：定义了很多变量时，需要对其中一部分变量进行操作时，可以使用tupledict的内置方法来高效构建表达式，比如对部分变量的求和，prod对相同变量下标的元素相乘等。

比如创建一个$3 \times 3$的矩阵，里面的每个元素表示线性表达式的变量。

$$
\left[\begin{array}{lll}
x_{11} & x_{12} & x_{13} \\
x_{21} & x_{22} & x_{23} \\
x_{31} & x_{32} & x_{33}
\end{array}\right]
$$

In [7]:
model = grb.Model()

# 定义变量的下标
t1 = [(1, 1), (1, 2), (1, 3),
      (2, 1), (2, 2), (2, 3),
      (3, 1), (3, 2), (3, 3)]

vars = model.addVars(t1, name='x')
model.update() # 添加完了变量之后要update一下。
# 基于元素下标的操作。
print(sum(vars.select(1, '*')))

Restricted license - for non-production use only - expires 2022-01-13
<gurobi.LinExpr: x[1,1] + x[1,2] + x[1,3]>


上述的代码相当于是对第一行求和，即$x_{11} + x_{12} + x_{13}$, 另一种写法为:

In [8]:
print(vars.sum(1, '*'))

<gurobi.LinExpr: x[1,1] + x[1,2] + x[1,3]>


上述的例子中，变量的系数都是1，如果变量的系数不是1的话，就不能用sum方法。而需要用prod方法，用于变量系数相乘后的累加。

In [9]:
c1 = [(1, 1), (1, 2), (1, 3)]
coeff = grb.tupledict(c1)
coeff[(1, 1)] = 1
coeff[(1, 2)] = 0.3
coeff[(1, 3)] = 0.4
print(vars.prod(coeff, 1, '*'))

<gurobi.LinExpr: x[1,1] + 0.3 x[1,2] + 0.4 x[1,3]>


In [10]:
m = grb.Model()
x = m.addVars(3, 4, vtype=grb.GRB.BINARY, name="x")
m.addConstrs((x.sum(i, '*') <= 1 for i in range(3)), name="con")
m.update()

m.write("tupledict_vars.lp")

# 创建的约束如下:

# con[0] : x[0, 0] + x[0, 1] + x[0, 2] + x[0, 3]
# con[1] : x[1, 0] + x[1, 1] + x[1, 2] + x[1, 3]
# con[2] : x[2, 0] + x[2, 1] + x[2, 2] + x[2, 3]

## 网络流例子

有两个城市，生产了两种商品(铅笔和钢笔)，必须装运到3个城市(波士顿、纽约和西雅图)的仓库，以满足给定的需求。

In [11]:
import gurobi as grb

In [12]:
commodities = ["Pencils", "Pens"] # 两种商品。
nodes = ['Detroit', 'Denver', 'Boston', 'New York', 'Seattle'] # 2个产地 + 3个目的地。

In [13]:
# 网络中每条弧的容量。

arcs, capacity = grb.multidict({
    ('Detroit', 'Boston'):   100,
    ('Detroit', 'New York'):  80,
    ('Detroit', 'Seattle'):  120,
    ('Denver',  'Boston'):   120,
    ('Denver',  'New York'): 120,
    ('Denver',  'Seattle'):  120})

In [14]:
# 商品在不同弧上的运输成本，是tupledict形式，可以用select，sum等加快变量选取。
cost = {
    ('Pencils', 'Detroit', 'Boston'):   10,
    ('Pencils', 'Detroit', 'New York'): 20,
    ('Pencils', 'Detroit', 'Seattle'):  60,
    ('Pencils', 'Denver',  'Boston'):   40,
    ('Pencils', 'Denver',  'New York'): 40,
    ('Pencils', 'Denver',  'Seattle'):  30,
    ('Pens',    'Detroit', 'Boston'):   20,
    ('Pens',    'Detroit', 'New York'): 20,
    ('Pens',    'Detroit', 'Seattle'):  80,
    ('Pens',    'Denver',  'Boston'):   60,
    ('Pens',    'Denver',  'New York'): 70,
    ('Pens',    'Denver',  'Seattle'):  30}

In [15]:
# 商品在不同节点的流入量，流出量，即需求量。   正数表示产地，负数表示需求量。
inflow = {
    ('Pencils', 'Detroit') : 50,
    ('Pencils', 'Denver') : 60,
    ('Pencils', 'Boston') : -50,
    ('Pencils', 'New York') : -50,
    ('Pencils', 'Seattle') : -10,
    
    ('Pens', 'Detroit') : 60,
    ('Pens', 'Denver') : 40,
    ('Pens', 'Boston') : -40,
    ('Pens', 'New York') : -30,
    ('Pens', 'Seattle') : -30,
}

In [16]:
m = grb.Model('netflow')

之后的话，我们就可以创建变量flow了。flow是tupledict类型的变量。 

键是('Pencils', 'Detroit', 'Boston')格式，表示的是某种商品，从某个产地到目的地的需求量。

In [17]:
flow = m.addVars(commodities, arcs, obj=cost, name="flow")
# obj (可自行选择设置): 决策变量在目标函数中的因子系数
# vtype (可自行选择设置): 决策变量类型，包括GRB.CONTINUOUS， GRB.BINARY， GRB.INTEGER，GRB.SEMICONT，GRB.SEMIINT

**容量限制**:

`i->j`, 产地到目的地，的所有不同商品的总量求和，应该是需要小于其对应的容量的限制。

In [18]:
m.addConstrs((flow.sum('*', i, j) <= capacity[i, j] for i, j in arcs), 'cap')

# Equivalent version using Python looping
# for i, j in arcs:
#     m.addConstr(sum(flow[h, i, j] for h in commodities) <= capacity[i, j],
#               "cap[%s, %s]" % (i, j))

{('Detroit', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Detroit', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Detroit', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>,
 ('Denver', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Denver', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Denver', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>}

**产地和目的地节点的流入、流出限制**

流入-流出=需求。

因为需求里面正数表示产地，负数表示需求量，所以：

流入 + 需求 = 流出

In [19]:
m.addConstrs((flow.sum(h, '*', j) + inflow[h, j] == flow.sum(h, j, '*')
        for h in commodities for j in nodes), "node")

# Alternate version:
# m.addConstrs(
#   (gp.quicksum(flow[h, i, j] for i, j in arcs.select('*', j)) + inflow[h, j] ==
#     gp.quicksum(flow[h, j, k] for j, k in arcs.select(j, '*'))
#     for h in commodities for j in nodes), "node")

{('Pencils', 'Detroit'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'Denver'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Detroit'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Denver'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>}

In [20]:
m.update()

In [21]:
m.optimize()
# m.computeIIS()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 16 rows, 12 columns and 36 nonzeros
Model fingerprint: 0xc43e5943
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]
Presolve removed 16 rows and 12 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.5000000e+03   0.000000e+00   2.000000e+01      0s
Extra simplex iterations after uncrush: 1
       1    5.5000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.03 seconds
Optimal objective  5.500000000e+03


In [22]:
# Print solution
if m.status == grb.GRB.OPTIMAL:
    solution = m.getAttr('x', flow)
    print(solution)
    for h in commodities:
        print('\nOptimal flows for %s:' % h)
        for i, j in arcs:
            if solution[h, i, j] > 0:
                print('%s -> %s: %g' % (i, j, solution[h, i, j]))

{('Pencils', 'Detroit', 'Boston'): 50.0, ('Pencils', 'Detroit', 'New York'): 0.0, ('Pencils', 'Detroit', 'Seattle'): 0.0, ('Pencils', 'Denver', 'Boston'): 0.0, ('Pencils', 'Denver', 'New York'): 50.0, ('Pencils', 'Denver', 'Seattle'): 10.0, ('Pens', 'Detroit', 'Boston'): 30.0, ('Pens', 'Detroit', 'New York'): 30.0, ('Pens', 'Detroit', 'Seattle'): 0.0, ('Pens', 'Denver', 'Boston'): 10.0, ('Pens', 'Denver', 'New York'): 0.0, ('Pens', 'Denver', 'Seattle'): 30.0}

Optimal flows for Pencils:
Detroit -> Boston: 50
Denver -> New York: 50
Denver -> Seattle: 10

Optimal flows for Pens:
Detroit -> Boston: 30
Detroit -> New York: 30
Denver -> Boston: 10
Denver -> Seattle: 30
